In [1]:
import pandas as pd 
import numpy as np 
import joblib 
import cv2 
from ultralytics import YOLO 
import torch 
from skimage.feature import hog



class OCRModel:
    def __init__(self):
        self.model1 = YOLO("yolov8n.pt")
        self.model1.model = torch.load("./model/yolo_char_rec.pt")
        self.model2 = joblib.load("./model/lreg_char_class.pkl")
        
    def prepare_img(self, img, xmin: float, ymin: float, xmax: float, ymax: float):
        # crop image to bounding box
        char_img = img[int(ymin):int(ymax), int(xmin):int(xmax)]

        # scale image
        char_img = cv2.resize(char_img, (64, 64))
        #plt.figure()
        hog_img = hog(char_img, orientations=9, pixels_per_cell=(4,4), cells_per_block=(1, 1))

        return hog_img
    
        
    def predict(self, image_path):
        results = self.model1(image_path)
        pred = []
        
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        for result in results[0].boxes.xyxy:
            xmin, ymin, xmax, ymax = result[0:4]
            char_img = self.prepare_img(img, xmin, ymin, xmax, ymax)
            pred.append(self.model2.predict([char_img]))
        return pred

In [27]:
df = pd.read_csv("../DataPreProcessing/.out/license_plates.csv")
#df.drop("Unnamed: 0", axis = 1, inplace= True)
display(df)

,plate,filename,char0,coord0,char1,coord1,char2,coord2,char3,coord3,...,char23,coord23,char24,coord24,char25,coord25,char26,coord26,char27,coord27
0,MH15TC554,0000.xml,M,13-17-27-32,H,25-16-38-33,1,36-16-46-34,5,45-16-55-33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MH02CB4545,0001.xml,M,15-23-32-41,H,33-26-46-44,0,51-29-64-47,2,63-32-77-50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KA031351,0003.xml,K,19-11-30-29,A,31-13-40-29,0,40-13-52-30,3,51-15-62-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TN0600000,0004.xml,T,156-27-228-156,N,248-31-322-159,0,362-32-435-159,6,456-38-526-161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KA51MJ8156,0005.xml,K,22-13-33-34,A,34-13-45-34,5,45-12-58-34,1,59-14-69-34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,MH2OBN3525,0232.xml,M,25-25-40-45,H,42-25-56-45,2,59-25-71-45,O,71-25-88-45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205,MH01AR5274,0233.xml,M,21-19-31-29,H,30-19-39-28,0,43-20-51-28,1,50-19-57-28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,MH20CS1941,0234.xml,M,18-18-31-35,H,32-18-42-35,2,43-19-54-34,0,54-18-64-34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,H20CS1938,0235.xml,H,21-15-27-26,2,27-16-32-26,0,32-17-37-27,C,37-16-43-28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
TrainingPartition = int(len(df) * 0.4) - 1 
ValidationPartition = int(len(df) * 0.7) - 1 
print("index of Training Partition is:", TrainingPartition) 
print("index of Valiadtion Partition is:", ValidationPartition) 

index of Training Partition is: 82
index of Valiadtion Partition is: 145


In [7]:
from torchmetrics.text import CharErrorRate
import os

images_path = "./data/LP-characters/images"

ImageFiles = [os.path.join(images_path, file)
                   for file in os.listdir(images_path)] # get the file content. TestingDataList = ImageFiles[144:]
ImageFiles = sorted(ImageFiles)
TestingDataList = ImageFiles[144:]
print(TestingDataList)

['B:\\projects\\uconn\\CSE5717\\group_proj\\CSE5717-Project\\OCR\\data\\LP-characters\\images\\0162.png', 'B:\\projects\\uconn\\CSE5717\\group_proj\\CSE5717-Project\\OCR\\data\\LP-characters\\images\\0163.png', 'B:\\projects\\uconn\\CSE5717\\group_proj\\CSE5717-Project\\OCR\\data\\LP-characters\\images\\0164.png', 'B:\\projects\\uconn\\CSE5717\\group_proj\\CSE5717-Project\\OCR\\data\\LP-characters\\images\\0167.png', 'B:\\projects\\uconn\\CSE5717\\group_proj\\CSE5717-Project\\OCR\\data\\LP-characters\\images\\0168.png', 'B:\\projects\\uconn\\CSE5717\\group_proj\\CSE5717-Project\\OCR\\data\\LP-characters\\images\\0169.png', 'B:\\projects\\uconn\\CSE5717\\group_proj\\CSE5717-Project\\OCR\\data\\LP-characters\\images\\0170.png', 'B:\\projects\\uconn\\CSE5717\\group_proj\\CSE5717-Project\\OCR\\data\\LP-characters\\images\\0171.png', 'B:\\projects\\uconn\\CSE5717\\group_proj\\CSE5717-Project\\OCR\\data\\LP-characters\\images\\0172.png', 'B:\\projects\\uconn\\CSE5717\\group_proj\\CSE5717-Pro

In [8]:
reader = OCRModel() 

In [9]:
import easyocr 
reader2 = easyocr.Reader(['en'], gpu=True) # the reader that will evaluate through the license plate text. 

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [45]:
def predictions_custom(model): 
    PredictionsList = [] 
    for i in TestingDataList:
        x = model.predict(i)
        char = ""
        if len(x) == 1: 
            char = x[0][1]
            char = char.upper()
            char = char.replace("-","")
            char = char.replace(" ","")
        else:
            for j in x:
                print(j)
                char+= j[0] #j[0]
                char = char.upper()
                char = char.replace("-","")
                char = char.replace(" ","")
        PredictionsList.append(char)
    return PredictionsList


def predictions_ez(reader): 
    PredictionsList = [] 
    for i in TestingDataList:
        x = reader.readtext(i)
        char = ""
        if len(x) == 1: 
            char = x[0][1]
            char = char.upper()
            char = char.replace("-","")
            char = char.replace(" ","")
        else:
            for j in x:
                char+= j[1] #j[0]
                char = char.upper()
                char = char.replace("-","")
                char = char.replace(" ","")
        PredictionsList.append(char)
    return PredictionsList

In [37]:
preds_custom = predictions_custom(reader)


image 1/1 B:\projects\uconn\CSE5717\group_proj\CSE5717-Project\OCR\data\LP-characters\images\0162.png: 192x640 9 letters, 34.0ms
Speed: 957.2ms preprocess, 34.0ms inference, 6.0ms postprocess per image at shape (1, 3, 192, 640)

image 1/1 B:\projects\uconn\CSE5717\group_proj\CSE5717-Project\OCR\data\LP-characters\images\0163.png: 320x640 10 letters, 44.6ms
Speed: 3.0ms preprocess, 44.6ms inference, 6.0ms postprocess per image at shape (1, 3, 320, 640)


c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, 


image 1/1 B:\projects\uconn\CSE5717\group_proj\CSE5717-Project\OCR\data\LP-characters\images\0164.png: 224x640 10 letters, 40.1ms
Speed: 3.0ms preprocess, 40.1ms inference, 4.9ms postprocess per image at shape (1, 3, 224, 640)


c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, 


image 1/1 B:\projects\uconn\CSE5717\group_proj\CSE5717-Project\OCR\data\LP-characters\images\0167.png: 224x640 10 letters, 35.0ms
Speed: 3.0ms preprocess, 35.0ms inference, 4.0ms postprocess per image at shape (1, 3, 224, 640)

image 1/1 B:\projects\uconn\CSE5717\group_proj\CSE5717-Project\OCR\data\LP-characters\images\0168.png: 192x640 10 letters, 36.6ms
Speed: 2.0ms preprocess, 36.6ms inference, 5.0ms postprocess per image at shape (1, 3, 192, 640)


...

c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, 


image 1/1 B:\projects\uconn\CSE5717\group_proj\CSE5717-Project\OCR\data\LP-characters\images\0234.png: 256x640 10 letters, 39.0ms
Speed: 3.0ms preprocess, 39.0ms inference, 5.0ms postprocess per image at shape (1, 3, 256, 640)



c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, 

image 1/1 B:\projects\uconn\CSE5717\group_proj\CSE5717-Project\OCR\data\LP-characters\images\0235.png: 320x640 10 letters, 43.0ms
Speed: 4.0ms preprocess, 43.0ms inference, 3.5ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 B:\projects\uconn\CSE5717\group_proj\CSE5717-Project\OCR\data\LP-characters\images\0236.png: 224x640 10 letters, 42.0ms
Speed: 3.5ms preprocess, 42.0ms inference, 7.0ms postprocess per image at shape (1, 3, 224, 640)


c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\ranyk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, 

In [46]:
preds_ez = predictions_ez(reader2)

In [35]:
len(df["plate"][144:])

65

In [47]:
CharErrorRate()(preds_custom, df["plate"][144:])

tensor(0.7631)

In [48]:
CharErrorRate()(preds_ez, df["plate"][144:])

tensor(0.2687)